<a href="https://colab.research.google.com/github/abdullahyasser0/Food-Search-engine-/blob/main/Food_Search_engine_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#sub

In [ ]:
!pip install --quiet python-terrier


In [ ]:
import pyterrier as pt

import re
import os


In [ ]:
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

In [ ]:
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

#importing

In [ ]:
!pip install --quiet nltk
!pip install --quiet flask pyngrok
!pip install --quiet flask_ngrok

!pip install --quiet kaggle
!pip3 install --quiet neptune-client==1.2.0
!pip install --quiet opendatasets

In [ ]:
!pip install --ignore-installed blinker
!pip install git+https://github.com/experimaestro/experimaestro-ir.git --quiet
import torch

# Choose GPU as device to run the experiments on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#install the transformer library
!pip install transformers


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors, Word2Vec
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import tensorflow_hub as hub
import opendatasets as od
import tensorflow as tf
import pyterrier as pt
import pandas as pd
import numpy as np
import jnius
import nltk
import json
import time
import os
import re


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
elmo = hub.load("https://tfhub.dev/google/elmo/3")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Functions

In [ ]:
def read_csv_files(directory, dataframes):

    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):

            read_csv_files(item_path, dataframes)
        elif item.endswith('.csv'):

            newData = pd.read_csv(item_path)
            dataframes.append(newData)
            print("File:", item)
            print(item_path)


In [ ]:
def search(query, index):
    try:
        tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
        preQuery = preprocess(query)
        rm3 = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)
        rmq = tf_idf >> rm3
        expanded = rmq.search(query).iloc[0]['query']
        res = tf_idf.search(expanded)
        return res
        # result_merged = pd.DataFrame({'docno': res['docno'], 'score': res['score']})
        # result_merged['docno'] = result_merged['docno'].astype(str)
        # last_df_proc['docno']=last_df_proc['docno'].astype(str)
        # result_merged = result_merged.merge(last_df_proc, on="docno")[["docno", "score", "title"]]

        # result_merged = result_merged.sort_values(by="score", ascending=False)

        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # from transformers import AutoTokenizer, AutoModel
        # bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        # bert_model = AutoModel.from_pretrained("bert-base-uncased")
        # from xpmir.models import AutoModel
        # model = AutoModel.load_from_hf_hub("xpmir/monot5", as_instance=True)

        # output = model.rsv(query, result_merged["title"].values)
        # data = [(list(obj.document.items.values())[0].text, obj.score) for obj in output]
        # result2 = pd.DataFrame(data, columns=['document', "score"]).sort_values(by="score", ascending=False)

        # return result_merged['docno'].tolist()


    except IndexError:
        return None

In [ ]:
def elmo_berd_search(query, index):
    try:
        elmo_res = elmo_search(query,index)

        elmo_res['docno'] = elmo_res['docno'].astype(str)
        last_df_proc['docno']=last_df_proc['docno'].astype(str)

        elmo_res = elmo_res.merge(last_df_proc, on="docno")[["docno", "score", "title"]]

        elmo_res = elmo_res.sort_values(by="score", ascending=False)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        from transformers import AutoTokenizer, AutoModel
        bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        bert_model = AutoModel.from_pretrained("bert-base-uncased")
        from xpmir.models import AutoModel
        model = AutoModel.load_from_hf_hub("xpmir/monot5", as_instance=True)

        output = model.rsv(query, elmo_res["title"].values)
        data = [(list(obj.document.items.values())[0].text, obj.score) for obj in output]
        result2 = pd.DataFrame(data, columns=['document', "score"]).sort_values(by="score", ascending=False)

        return elmo_res['docno'].tolist()


    except IndexError:
        return None

In [ ]:
def elmo_search(query, index):
  result = search(query,index)
  result_data=[]
  result_data.append(query)
  index = []
  if result is None:
    print('')
  else:
      for i in range(len(result)):
          docno_index = int(result['docno'][i])
          if docno_index < len(last_df):
              result_data.append(last_df['title'][docno_index])
              index.append(docno_index)
          else:
              print("")

  embeddings = elmo.signatures["default"](tf.constant(result_data))["elmo"]
  sentence_embeddings = np.mean(embeddings.numpy(), axis=1)
  similarity_score=[]
  for i in range(1,len(result_data)-1):
    similarity_score.append([cosine_similarity(sentence_embeddings[0], sentence_embeddings[i]),index[i]])
  sorted_data = sorted(similarity_score, key=lambda x: x[0], reverse=True)

  column_names = ['score', 'docno']

  return pd.DataFrame(sorted_data, columns=column_names)




In [ ]:
elmo_berd_search("fried", index3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

path:   0%|          | 0.00/990M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 13.06 MiB is free. Process 2240 has 14.73 GiB memory in use. Of the allocated memory 926.44 MiB is allocated by PyTorch, and 87.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

In [ ]:
def Stem_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

def clean(text):
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=\@]", " ", text)
   text = re.sub(r'\t', ' ', text)
   text = re.sub(r'\n', ' ', text)
   text = re.sub(r"'", '', text)
   text = re.sub(r"\s+", " ", text)
   text = text.strip()
   return text

def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

def preprocess(sentence):
  sentence = clean(sentence)
  sentence = remove_stopwords(sentence)
  sentence = Stem_text(sentence)
  return sentence

In [ ]:
def html():
  import os

  if not os.path.exists("templates"):
      os.makedirs("templates")


  html_content = """
  <!DOCTYPE html>
  <html lang="en">
  <head>
      <meta charset="UTF-8">
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
      <title>Search</title>
      <style>
          body {
              font-family: Arial, sans-serif;
              background-color: #f4f4f4;
              margin: 0;
          }
          .container {
              max-width: 800px;
              margin: 0 auto;
              padding: 20px;
          }
          input[type="text"] {
              padding: 10px;
              width: 100%;
              font-size: 16px;
              border: 1px solid #ccc;
              border-radius: 5px;
              margin-bottom: 10px;
          }
          button {
              padding: 10px 20px;
              font-size: 16px;
              background-color: #4CAF50;
              color: white;
              border: none;
              border-radius: 5px;
              cursor: pointer;
          }
          button:hover {
              background-color: #45a049;
          }
          .search-results {
              text-align: left;
          }
          .search-result {
              margin-bottom: 20px;
          }
          .search-result p {
              margin: 0;
          }
          .search-result-title {
              font-weight: bold;
          }
          .search-result-snippet {
              margin-left: 20px;
              font-weight: normal; /* Adjust font weight here */
          }
          /* Smaller font size for time taken and number of results */
          .time-taken, .num-results {
              font-size: 12px;
              display: inline-block;
              margin-right: 10px;
          }
          .error-message {
              color: red;
              font-weight: bold;
              margin-top: 10px;
          }
      </style>
  </head>
  <body>
      <div class="container">
          <h1> Hungry! The food hub is your home </h1>
          <form method="post">
              <input type="text" name="query" placeholder="search a dish" value="{{ query }}">
              <button type="submit">Search</button>
          </form>
          {% if error_message %}
              <p class="error-message">{{ error_message }}</p>
          {% endif %}
          {% if time_taken is defined %}
              <p class="time-taken">Time taken: {{ time_taken }} seconds</p>
          {% endif %}
          {% if num_results is defined %}
              <p class="num-results">Number of results: {{ num_results }}</p>
          {% endif %}

          {% if results %}
              <h2>Search Results</h2>
              <div class="search-results">
                  {% for result in results %}
                      <div class="search-result">
                          <p class="search-result-title"><a href="{{ result['link'] }}" target="_blank">{{ result['title'] }}</a></p>
                          <p class="search-result-snippet">{{ result['snipped'] }}</p>
                      </div>
                  {% endfor %}
              </div>
          {% endif %}
      </div>
  </body>
  </html>




  """

  # Write the content to the index.html file
  with open("templates/index.html", "w") as f:
      f.write(html_content)

#**DATA IMPORTING**

In [ ]:
with open('kaggle.json', 'w') as f:
        json.dump({"username":"abdlllahyasser","key":"896a54a3fb943ab27ee088c15ee6443e"}, f)

In [ ]:
od.download('https://www.kaggle.com/datasets/eliasdabbas/recipes-search-engine-results-data')

Dataset URL: https://www.kaggle.com/datasets/eliasdabbas/recipes-search-engine-results-data


100%|██████████| 6.56M/6.56M [00:00<00:00, 135MB/s]

In [ ]:
last_df=pd.read_csv('./recipes-search-engine-results-data/recipes_serp_data.csv')
last_df_proc=last_df.copy()

# **preproceses and indexing**

In [ ]:
last_df_proc = last_df_proc.drop_duplicates(subset='title')
last_df_proc['docno'] = last_df_proc.index

In [ ]:
last_df_proc['title']=last_df_proc['title'].astype(str).apply(preprocess)

In [ ]:
pd_indexer3 = pt.DFIndexer("./pd_index3",overwrite=True)
indexref3 = pd_indexer3.index(last_df_proc["title"].astype(str), last_df_proc["docno"].astype(str))
index3 = pt.IndexFactory.of(indexref3)

22:22:04.362 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (803) - further warnings are suppressed
22:22:06.632 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents


#**Test**

In [ ]:
import time

now = time.time()
search_results = search("dish recipe	", index3)
then = time.time()

if search_results is None:
    print("Sorry, no results found.")
else:
    print("Time taken is:", then - now)
    print("Number of results:", len(search_results))
    for i in range(min(10, len(search_results))):
        docno_index = int(search_results['docno'][i])
        if docno_index < len(last_df):
            print(last_df['title'][docno_index], last_df['link'][docno_index])
        else:
            print("Invalid index:", docno_index, "- Out of bounds")


Time taken is: 0.3420703411102295
Number of results: 1000
Recipes - Dish https://dish.co.nz/recipes/
National Dish of Kazakhstan Beshbarmak – National Dishes of the ... https://nationalfoods.org/recipe/national-dish-of-kazakhstan-beshbarmak/
Recipes | The Mediterranean Dish https://www.themediterraneandish.com/recipes/
Ambuyat National Dish of Brunei – National Dishes of the World https://nationalfoods.org/recipe/ambuyat-national-dish-of-brunei/
National Dish of Eritrea – Zigini with Injera – National Dishes of the ... https://nationalfoods.org/recipe/national-dish-of-eritrea-zigini-with-injera/
National Dish of Gabon – Poulet Nyembwe – National Dishes of the ... https://nationalfoods.org/recipe/national-dish-of-gabon-poulet-nyembwe/
National Dish of Liberia Dumboy – National Dishes of the World https://nationalfoods.org/recipe/national-dish-of-liberia-dumboy/
National Dish of Gambia – Benachin – National Dishes of the World https://nationalfoods.org/recipe/national-dish-of-gambia-bena

In [ ]:
import time

now = time.time()
search_results = search("dish recipe	", index3)
then = time.time()
if search_results is None:
    print("Sorry, no results found.")
else:
    print("Time taken is:", then - now)
    print("Number of results:", len(search_results))
    for i in range(min(10, len(search_results))):
      x=int(search_results[i])
      print(last_df['title'][x], last_df['link'][x])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

path:   0%|          | 0.00/990M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

#**Evaluation**

In [ ]:
queries=last_df['searchTerms'].unique()
resultedmatch=[]
for i in range(len(queries)):
    try:
        ref = set(last_df[last_df['searchTerms'] == queries[i]]['title'])
        search_results = search(queries[i], index3)
        if search_results is None:
            s_res = []
        else:
            for i in range(10):
              docno_index = int(search_results['docno'][i])
              if docno_index < len(last_df):
                s_res.append(last_df['title'][docno_index])
            set2=set(s_res)
            resultedmatch.append(len(ref.intersection(set2)))
            pass
    except jnius.JavaException as e:
        continue

In [ ]:
evaluation_percent=(sum(resultedmatch)/len(resultedmatch)*10 )

In [ ]:
evaluation_percent

#**Interface**

In [ ]:
html()

In [ ]:
from google.colab.output import eval_js
print (eval_js("google.colab.kernel.proxyPort(5000)"))

https://c6dkc6twuno-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
import time

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/", methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        query = request.form['query']
        start_time = time.time()
        search_results = search(query, index3)
        end_time = time.time()

        if search_results is None:
            error_message = "Sorry, no results found."
            return render_template('index.html', error_message=error_message)

        result_data = []
        for i in range(len(search_results['docno'])):
            result_data.append({
                'title': last_df['title'][int(search_results['docno'][i])],
                'link': last_df['link'][int(search_results['docno'][i])],
                'snipped' : last_df['snippet'][int(search_results['docno'][i])]
            })

        time_taken = end_time - start_time
        num_results = len(search_results['docno'])

        return render_template('index.html', query=query, results=result_data, time_taken=time_taken, num_results=num_results)

    return render_template('index.html')

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/May/2024 20:03:52] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 20:03:53] "GET /favicon.ico HTTP/1.1" 404 -
Exception in thread Thread-26:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/py

#Trash

In [ ]:
paths=[ 'https://www.kaggle.com/datasets/brianblakely/top-100-songs-and-lyrics-from-1959-to-2019',
        'https://www.kaggle.com/datasets/ceebloop/rap-lyrics-for-nlp',
        'https://www.kaggle.com/datasets/abdelrahmanekhaldi/english-poem-dataset',
        'https://www.kaggle.com/datasets/terminate9298/songs-lyrics',
        'https://www.kaggle.com/datasets/joebeachcapital/57651-spotify-songs'

       ]

In [ ]:
for link in paths:
  od.download(link)

In [ ]:
filenames=[]
for path in paths:
    filenames.append(path.split('/')[-1])

In [ ]:
df1=pd.read_csv('/content/57651-spotify-songs/Spotify Million Song Dataset_exported.csv')
df2=pd.read_csv('/content/english-poem-dataset/PoemDataset.csv')
df3=pd.read_csv('/content/rap-lyrics-for-nlp/lyrics_raw.csv')
df4=pd.read_csv('/content/songs-lyrics/lyrics.csv')
df5=pd.read_csv('/content/top-100-songs-and-lyrics-from-1959-to-2019/all_songs_data.csv')

In [ ]:
dataframes = []
read_csv_files('.', dataframes)

PREPROCES

In [ ]:
df4

In [ ]:
df1.rename(columns={'song':'title','artist':'artist','text':'lyrics'},inplace=True)
df1.drop(columns='link',inplace=True)

df3.rename(columns={'track_name':'title','artist':'artist','raw_lyrics':'lyrics'},inplace=True)
df3.drop(columns='artist_verses',inplace=True)

df2.rename(columns={'Title':'title','Poet':'artist','Poem':'lyrics'},inplace=True)
df2.drop(columns='Genre',inplace=True)

df4.rename(columns={'song_name':'title','artist':'artist','lyrics':'lyrics'},inplace=True)
df4.drop(columns={'link','Unnamed: 0'},inplace=True)

df5.rename(columns={'Song Title':'title','Artist':'artist','Lyrics':'lyrics'},inplace=True)
df5.drop(columns={'Album URL','Featured Artists','Media','Rank','Album','Song URL','Writers','Year','Release Date'},inplace=True)

In [ ]:
readyFrames=[df1,df2,df3,df4,df5]
resultDF = pd.concat(readyFrames, ignore_index=True)
resultDF=resultDF.drop_duplicates(subset=['title'])
resultDF=resultDF.reset_index()
preprocDf=resultDF.copy()
preprocDf['docno'] = preprocDf.index

In [ ]:
preprocDf

In [ ]:
preprocDf['lyrics']=preprocDf['lyrics'].astype(str).apply(preprocess)

In [ ]:
preprocDf['title']=preprocDf['title'].astype(str).apply(preprocess)

DATASET plan B

In [ ]:
links =['https://www.kaggle.com/datasets/bhaveshmittal/python-programming-questions-dataset/data'

]

In [ ]:
for link in links:
  od.download(link)

In [ ]:
bDf=pd.read_csv('/content/python-programming-questions-dataset/Python Programming Questions Dataset.csv')
bDfpro=bDf.copy()

In [ ]:
bDf

In [ ]:
bDfpro['Instruction']=bDfpro['Instruction'].astype(str).apply(preprocess)

In [ ]:
bDfpro['docno'] = bDfpro.index

In [ ]:
pd_indexer1 = pt.DFIndexer("./pd_index1",overwrite=True)
indexref1 = pd_indexer1.index(bDfpro["Instruction"].astype(str), bDfpro["docno"].astype(str))
index1 = pt.IndexFactory.of(indexref1)

In [ ]:
search_results = search("bubble sort",index1)
for i in range (10):
  print ("function number >> ",i)
  print(bDf['Output'][int(search_results['docno'][i])])

indexing

In [ ]:
pd_indexer = pt.DFIndexer("./pd_index",overwrite=True)
indexref = pd_indexer.index(preprocDf["lyrics"].astype(str), preprocDf["docno"].astype(str))
index = pt.IndexFactory.of(indexref)

In [ ]:
pd_indexer2 = pt.DFIndexer("./pd_index2",overwrite=True)
indexref2 = pd_indexer2.index(preprocDf["title"].astype(str), preprocDf["docno"].astype(str))
index2 = pt.IndexFactory.of(indexref2)

In [ ]:
now=time.time()
search_results = search("fly me to the moon",index)
search_results1=search("fly me to the moon",index2)
then=time.time()

print ("Time taken is : ", then-now)
print("Number of results : ", len(search_results))
for i in range (10):
  print(resultDF['title'][int(search_results['docno'][i])],", artist : ", resultDF['artist'][int(search_results['docno'][i])])
  print(resultDF['title'][int(search_results1['docno'][i])],", artist : ", resultDF['artist'][int(search_results1['docno'][i])])